In [1]:
import os
!pwd

/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Text_summarzer_project/research


In [2]:
os.chdir('../')
!pwd

/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Text_summarzer_project


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [16]:
from textSummarizer.constants import *
from textSummarizer.utils.utils import read_yaml, create_directories
# from textSummarizer.entity import *

class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        with open(self.config.data_validation.STATUS_FILE, 'r') as f:
            status = f.read()
        
        if status:
            config = self.config.data_transformation

            create_directories([config.root_dir])

            data_transformation_config = DataTransformationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                tokenizer_name=config.tokenizer_name
            )

            return data_transformation_config

In [17]:
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length=1024,
            truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'],
                max_length=128,
                truncation=True
            )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2024-02-07 21:03:54,393: INFO: utils: yaml file config/config.yaml loaded successfully:]
[2024-02-07 21:03:54,394: INFO: utils: yaml file params.yaml loaded successfully:]
[2024-02-07 21:03:54,395: INFO: utils: created directory at: artifacts:]
[2024-02-07 21:03:54,398: INFO: utils: created directory at: artifacts/data_transformation:]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 110497.28 examples/s]
